## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-12-05-22-52-36 +0000,eltecolote,9 free food pantries in San Francisco offering...,https://eltecolote.org/content/en/sf-free-food...
1,2025-12-05-22-49-32 +0000,nyt,"Robert B. Fiske Jr., First to Lead Whitewater ...",https://www.nytimes.com/2025/12/05/us/robert-b...
2,2025-12-05-22-46-00 +0000,wsj,Opinion | Trump and a Mozambique LNG Boondoggle,https://www.wsj.com/opinion/totalenergies-moza...
3,2025-12-05-22-45-52 +0000,wapo,Hong Kong tycoon Jimmy Lai’s health is declini...,https://www.washingtonpost.com/national-securi...
4,2025-12-05-22-45-47 +0000,nyt,"Chuck Kesey, Probiotic Yogurt Pioneer, Dies at 87",https://www.nytimes.com/2025/12/05/dining/chuc...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2375/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
16,trump,50
48,new,22
66,court,18
91,supreme,15
165,netflix,13


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
224,2025-12-05-11-42-00 +0000,wsj,The Supreme Court ruled that Texas could legal...,https://www.wsj.com/politics/policy/a-huge-red...,123
100,2025-12-05-19-14-25 +0000,latimes,Supreme Court will hear Trump's plan to restri...,https://www.latimes.com/politics/story/2025-12...,116
136,2025-12-05-17-49-24 +0000,nypost,President Trump honored as ‘very first winner’...,https://nypost.com/2025/12/05/us-news/presiden...,113
345,2025-12-04-23-23-00 +0000,wsj,The Trump administration intends to formally s...,https://www.wsj.com/politics/policy/white-hous...,110
214,2025-12-05-12-19-01 +0000,cbc,U.S. Supreme Court paves way for midterm redis...,https://www.cbc.ca/news/world/us-scotus-texas-...,109


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
224,123,2025-12-05-11-42-00 +0000,wsj,The Supreme Court ruled that Texas could legal...,https://www.wsj.com/politics/policy/a-huge-red...
211,76,2025-12-05-12-33-52 +0000,latimes,Netflix agrees to buy Warner Bros. in an $82.7...,https://www.latimes.com/entertainment-arts/bus...
318,68,2025-12-05-00-22-00 +0000,wsj,"In a briefing for lawmakers, Navy Adm. Frank “...",https://www.wsj.com/politics/national-security...
154,63,2025-12-05-16-51-04 +0000,nypost,DC pipe-bomb suspect Brian Cole Jr. appears in...,https://nypost.com/2025/12/05/us-news/accused-...
175,48,2025-12-05-15-35-55 +0000,nypost,AG Bondi rips Minnesota for allowing Somali mi...,https://nypost.com/2025/12/05/us-news/pam-bond...
51,47,2025-12-05-21-13-25 +0000,nyt,Gianni Infantino Awards Trump a FIFA Peace Pri...,https://www.nytimes.com/2025/12/05/us/politics...
176,42,2025-12-05-15-35-40 +0000,bbc,'I don't want to be part of this war machine':...,https://www.bbc.com/news/videos/cx25031x23ro?a...
160,41,2025-12-05-16-19-00 +0000,wsj,A key vaccine panel at the Centers for Disease...,https://www.wsj.com/politics/policy/cdc-panel-...
177,40,2025-12-05-15-33-01 +0000,nypost,Trump national security strategy calls out ‘an...,https://nypost.com/2025/12/05/us-news/trump-na...
75,34,2025-12-05-20-18-48 +0000,nyt,Judge Approves Release of Epstein Grand Jury D...,https://www.nytimes.com/2025/12/05/us/politics...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
